In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re, pip, conda

# 二 贝叶斯优化的实现

贝叶斯优化是当今黑盒函数估计领域最为先进和经典的方法，在同一套序贯模型下使用不同的代理模型以及采集函数、还可以发展出更多更先进的贝叶斯优化改进版算法，因此，贝叶斯优化的其算法本身就多如繁星，实现各种不同种类的贝叶斯优化的库也是琳琅满目，几乎任意一个专业用于超参数优化的工具库都会包含贝叶斯优化的内容。我们可以在以下页面找到大量可以实现贝叶斯优化方法的HPO库：https://www.automl.org/automl/hpo-packages/ ，其中大部分库都是由独立团队开发和维护，因此不同的库之间之间的优劣、性格、功能都有很大的差异。在课程中，我们将介绍如下三个可以实现贝叶斯优化的库：`bayesian-optimization`，`hyperopt`，`optuna`。

|HPO库|优劣评价|推荐指数|
|-|-|-|
|**bayes_opt**|✅实现基于高斯过程的贝叶斯优化<br>✅当参数空间由大量连续型参数构成时<br><br>⛔包含大量离散型参数时避免使用<br>⛔算力/时间稀缺时避免使用|⭐⭐|
|**hyperopt**|✅实现基于TPE的贝叶斯优化<br>✅支持各类提效工具<br>✅进度条清晰，展示美观，较少怪异警告或报错<br>✅可推广/拓展至深度学习领域<br><br>⛔不支持基于高斯过程的贝叶斯优化<br>⛔代码限制多、较为复杂，灵活性较差|⭐⭐⭐⭐|
|**optuna**|✅（可能需结合其他库）实现基于各类算法的贝叶斯优化<br>✅代码最简洁，同时具备一定的灵活性<br>✅可推广/拓展至深度学习领域<br><br>⛔非关键性功能维护不佳，有怪异警告与报错|⭐⭐⭐⭐|

注意，以上三个库<font color="red">**都不支持基于Python环境的并行或加速**</font>，大多数优化算法库只能够支持基于数据库（如MangoDB，mySQL）的并行或加速，但以上库都可以被部署在分布式计算平台。

三个库极其辅助包的安装方法分别如下，使用pip或conda安装时注意关闭梯子。

- Bayes_opt

In [ ]:
#!pip install bayesian-optimization
#!conda install -c conda-forge bayesian-optimization

- Hyperopt

In [ ]:
#!pip install hyperopt

- Optuna

In [ ]:
#!pip install optuna
#!conda install -c conda-forge optuna

- Skopt（作为Optuna辅助包安装，也可单独使用）

In [ ]:
#!pip install scikit-optimize

接下来我们会分别使用三个库来实现贝叶斯优化。在课程中，我们依然使用集成算法中的房价数据作为验证数据，并且呈现出我们之前在不同优化方法上得出的结果作为对比。同时，我们将使用与集成算法中完全一致的随机数种子、以及随机森林算法作为被优化的评估器。

- **导入库，确认使用数据**

In [4]:
#基本工具
import numpy as np
import pandas as pd
import time
import os #修改环境设置

#算法/损失/评估指标等
import sklearn
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import KFold, cross_validate

#优化器
from bayes_opt import BayesianOptimization

import hyperopt
from hyperopt import hp, fmin, tpe, Trials, partial
from hyperopt.early_stop import no_progress_loss

import optuna

Bayes_opt版本：1.2.0

In [5]:
print(optuna.__version__)

2.10.0


In [6]:
print(hyperopt.__version__)

0.2.7


In [7]:
data = pd.read_csv(r"D:\Pythonwork\2021ML\PART 2 Ensembles\datasets\House Price\train_encode.csv",index_col=0)

X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [8]:
X.head()

,Id,住宅类型,住宅区域,街道接触面积(英尺),住宅面积,街道路面状况,巷子路面状况,住宅形状(大概),住宅现状,水电气,...,半开放式门廊面积,泳池面积,泳池质量,篱笆质量,其他配置,其他配置的价值,销售月份,销售年份,销售类型,销售状态
0,0.0,5.0,3.0,36.0,327.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,8.0,4.0
1,1.0,0.0,3.0,51.0,498.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,8.0,4.0
2,2.0,5.0,3.0,39.0,702.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,8.0,4.0
3,3.0,6.0,3.0,31.0,489.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.0,0.0
4,4.0,5.0,3.0,55.0,925.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,2.0,8.0,4.0


In [9]:
X.shape

(1460, 80)

- **确认该数据集上的历史成果**

|HPO方法|默认参数|网格搜索|随机搜索|随机搜索<br>(大空间)|随机搜索<br>(连续型)|
|:-:|:-:|:-:|:-:|:-:|:-:|
|搜索空间/全域空间|-|1536/1536|800/1536|1536/3000|1536/无限|
|运行时间（分钟）|-|6.36|<font color="green">**2.83(↓)**</font>|<font color="green">**3.86(↓)**</font>|3.92|
|搜索最优（RMSE）|30571.266|29179.698|29251.284|<font color="green">**29012.905(↓)**</font>|29148.381|
|重建最优（RMSE）|-|28572.070|<font color="brown">**28639.969(↑)**</font>|<font color="green">**28346.673(↓)**</font>|28495.682|

## 3 基于Optuna实现多种贝叶斯优化

Optuna是目前为止最为成熟、拓展性最强的超参数优化框架，与古旧的bayes_opt相比，Optuna明显是专门为机器学习和深度学习所设计。为了满足机器学习开发者的需求，Optuna拥有强大且固定的API，因此Optuna代码简单，编写高度模块化，是我们介绍的库中代码最为简练的库。Optuna的优势在于，它可以无缝衔接到PyTorch、Tensorflow等深度学习框架上，也可以与sklearn的优化库scikit-optimize结合使用，因此Optuna可以被用于各种各样的优化场景。在我们的课程中，我们将重点介绍Optuna实现贝叶斯优化的过程，其他优化方面内容可以参考以下页面：https://github.com/optuna/optuna 。

In [25]:
import optuna

In [26]:
print(optuna.__version__)

2.10.0


- 1 定义目标函数与参数空间

Optuna的目标函数相当特别。在其他优化库中，我们需要单独输入参数或参数空间，优化器会在具体优化过程中将参数空间一一放入我们的目标函数进行优化，但在Optuna中，我们并不需要将参数或参数空间输入目标函数，而是需要**直接在目标函数中定义参数空间**。特别的是，Optuna优化器会生成一个指代备选参数的变量trial，该变量无法被用户获取或打开，但该变量在优化器中生存，并被输入目标函数。在目标函数中，我们可以通过变量trail所携带的方法来构造参数空间，具体如下所示：

In [27]:
def optuna_objective(trial):
    
    #定义参数空间
    n_estimators = trial.suggest_int("n_estimators",80,100,1) #整数型，(参数名称，下界，上界，步长)
    max_depth = trial.suggest_int("max_depth",10,25,1)
    max_features = trial.suggest_int("max_features",10,20,1)
    #max_features = trial.suggest_categorical("max_features",["log2","sqrt","auto"]) #字符型
    min_impurity_decrease = trial.suggest_int("min_impurity_decrease",0,5,1)
    #min_impurity_decrease = trial.suggest_float("min_impurity_decrease",0,5,log=False) #浮点型
    
    #定义评估器
    #需要优化的参数由上述参数空间决定
    #不需要优化的参数则直接填写具体值
    reg = RFR(n_estimators = n_estimators
              ,max_depth = max_depth
              ,max_features = max_features
              ,min_impurity_decrease = min_impurity_decrease
              ,random_state=1412
              ,verbose=False
              ,n_jobs=-1
             )
    
    #交叉验证过程，输出负均方根误差(-RMSE)
    #optuna同时支持最大化和最小化，因此如果输出-RMSE，则选择最大化
    #如果选择输出RMSE，则选择最小化
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(reg,X,y
                                     ,scoring="neg_root_mean_squared_error"
                                     ,cv=cv #交叉验证模式
                                     ,verbose=False #是否打印进程
                                     ,n_jobs=-1 #线程数
                                     ,error_score='raise'
                                    )
    #最终输出RMSE
    return np.mean(abs(validation_loss["test_score"]))

- 2 定义优化目标函数的具体流程

在HyperOpt当中我们可以调整参数`algo`来自定义用于执行贝叶斯优化的具体算法，在Optuna中我们也可以。大部分备选的算法都集中在Optuna的模块sampler中，包括我们熟悉的TPE优化、随机网格搜索以及其他各类更加高级的贝叶斯过程，对于Optuna.sampler中调出的类，我们也可以直接输入参数来设置初始观测值的数量、以及每次计算采集函数时所考虑的观测值量。在Optuna库中并没有集成实现高斯过程的方法，但我们可以从scikit-optimize里面导入高斯过程来作为optuna中的`algo`设置，而具体的高斯过程相关的参数则可以通过如下方法进行设置：

In [28]:
def optimizer_optuna(n_trials, algo):
    
    #定义使用TPE或者GP
    if algo == "TPE":
        algo = optuna.samplers.TPESampler(n_startup_trials = 10, n_ei_candidates = 24)#n_startup_trials初始观测值,n_ei_candidates采集函数
    elif algo == "GP":
        from optuna.integration import SkoptSampler
        import skopt
        algo = SkoptSampler(skopt_kwargs={'base_estimator':'GP', #选择高斯过程
                                          'n_initial_points':10, #初始观测点10个
                                          'acq_func':'EI'} #选择的采集函数为EI，期望增量
                           )
    
    #实际优化过程，首先实例化优化器
    study = optuna.create_study(sampler = algo #要使用的具体算法
                                , direction="minimize" #优化的方向，可以填写minimize或maximize,控制输出的最大还是最小,如果是rmse则是最小,如果是r2则是最大
                               )
    #开始优化，n_trials为允许的最大迭代次数
    #由于参数空间已经在目标函数中定义好，因此不需要输入参数空间
    study.optimize(optuna_objective #目标函数
                   , n_trials=n_trials #最大迭代次数（包括最初的观测值的）
                   , show_progress_bar=True #要不要展示进度条呀？
                  )
    
    #可直接从优化好的对象study中调用优化的结果
    #打印最佳参数与最佳损失值
    print("\n","\n","best params: ", study.best_trial.params,
          "\n","\n","best score: ", study.best_trial.values,
          "\n")
    
    return study.best_trial.params, study.best_trial.values

- 3 执行实际优化流程

Optuna库虽然是当今最为成熟的HPO方法之一，但当参数空间较小时，Optuna库在迭代中容易出现抽样BUG，**即Optuna会持续抽到曾经被抽到过的参数组合**，并且持续报警告说"算法已在这个参数组合上检验过目标函数了"。在实际迭代过程中，一旦出现这个Bug，那当下的迭代就无用了，因为已经检验过的观测值不会对优化有任何的帮助，因此对损失的优化将会停止。如果出现该BUG，则可以增大参数空间的范围或密度。或者使用如下的代码令警告关闭：

In [ ]:
import warnings
warnings.filterwarnings('ignore', message='The objective has been evaluated at this point before.')

In [29]:
best_params, best_score = optimizer_optuna(10,"GP") #默认打印迭代过程

[I 2021-12-24 22:14:26,709] A new study created in memory with name: no-name-05950945-f6f7-41c3-bd8a-ffb15a284ea9
D:\ProgramData\Anaconda3\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2021-12-24 22:14:28,229] Trial 0 finished with value: 28848.70339210933 and parameters: {'n_estimators': 99, 'max_depth': 14, 'max_features': 16, 'min_impurity_decrease': 4}. Best is trial 0 with value: 28848.70339210933.
[I 2021-12-24 22:14:29,309] Trial 1 finished with value: 28632.395126147465 and parameters: {'n_estimators': 90, 'max_depth': 23, 'max_features': 16, 'min_impurity_decrease': 2}. Best is trial 1 with value: 28632.395126147465.
[I 2021-12-24 22:14:30,346] Trial 2 finished with value: 29301.159287113685 and parameters: {'n_estimators': 89, 'max_depth': 17, 'max_features': 12, 'min_impurity_decrease': 0}. Best is trial 1 with value: 28632.395126147465.
[I 2021-12-24 22:14:31,215] Trial 3 finished with value: 29756.446415640086 and parameters: {'n_estimators': 80, 'max_depth': 11, 'max_features': 14, 'min_impurity_decrease': 3}. Best is trial 1 with value: 28632.395126147465.
[I 2021-12-24 22:14:31,439] Trial 4 finished with value: 29784.547574554617 and parameters: {'

In [80]:
optuna.logging.set_verbosity(optuna.logging.ERROR) #关闭自动打印的info，只显示进度条
#optuna.logging.set_verbosity(optuna.logging.INFO)
best_params, best_score = optimizer_optuna(300,"TPE")

D:\ProgramData\Anaconda3\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/300 [00:00<?, ?it/s]


 
 best params:  {'n_estimators': 96, 'max_depth': 22, 'max_features': 14, 'min_impurity_decrease': 3} 
 
 best score:  [28457.22400533479] 



In [85]:
optuna.logging.set_verbosity(optuna.logging.ERROR)
best_params, best_score = optimizer_optuna(300,"GP")

D:\ProgramData\Anaconda3\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/300 [00:00<?, ?it/s]


 
 best params:  {'n_estimators': 87, 'max_depth': 23, 'max_features': 16, 'min_impurity_decrease': 5} 
 
 best score:  [28541.05837443567] 



很显然，基于高斯过程的贝叶斯优化是比基于TPE的贝叶斯优化运行更加缓慢的。在Optuna进行调试时，我并没有多次运行并取出Optuna表现最好的值，因此我们可以不将Optuna的结果最终放入表格进行比较，不过在TPE模式下，其运行速度与HyperOpt的运行速度高度接近。在未来的课程中，除非特殊说明，我们将默认使用TPE方法进行优化。